In [473]:
# For Data Manipulation
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
%matplotlib

Using matplotlib backend: MacOSX


In [504]:
# Load the data
data=pd.read_csv('data_cleaned_step_2.csv')

### Inspect the data

In [505]:
data.head()

,opportunity_id,application_term,application_received_date,file_status_completed_date,decision_date,deposit_received_date,response_date,date_acceptance_letter_sent,department,undergraduate_gpa,...,follow_up_required,90041_mi,91601_mi,92122_mi,94607_mi,last_60_90_gpa,graduate_gpa,temperature,second_review_lor,sip_score
0,006a000000zHwSbAAK,Fall 2013,2013-03-15T00:00:00Z,2013-05-13T00:00:00Z,2013-06-06T00:00:00Z,NaN,NaN,2013-06-07T00:00:00Z,blank,2.00,...,False,14.112139,17.312897,96.109994,351.040530,2.20,3.67,blank,3,0
1,006a000000zHwQgAAK,Fall 2013,2013-03-11T00:00:00Z,2013-07-09T00:00:00Z,2013-07-09T00:00:00Z,NaN,NaN,2013-05-17T00:00:00Z,blank,2.01,...,False,30.434642,36.394601,76.712759,370.439711,2.88,3.67,blank,3,0
2,006a000000yvNgDAAU,Fall 2013,2013-02-21T00:00:00Z,2013-03-15T00:00:00Z,2013-05-01T00:00:00Z,NaN,NaN,2013-05-03T00:00:00Z,blank,2.12,...,False,13.940513,14.910570,99.376021,347.804562,2.94,3.09,blank,2,0
3,006a000001BhqChAAJ,Fall 2014,2014-12-09T00:00:00Z,2014-12-09T00:00:00Z,2015-03-20T00:00:00Z,NaN,NaN,NaN,blank,1.70,...,False,2319.139657,2328.959417,2257.727213,2567.392367,2.88,3.67,blank,3,0
4,006a0000017FtkIAAS,Summer 2014,2014-02-13T00:00:00Z,2014-02-13T00:00:00Z,2014-04-24T00:00:00Z,NaN,NaN,2014-05-09T00:00:00Z,blank,1.95,...,False,NaN,NaN,NaN,NaN,2.25,3.67,blank,1,0


### Deal with date variables

In [506]:
# Convert Strings to Date
data.application_received_date=pd.to_datetime(data.application_received_date)
data.file_status_completed_date=pd.to_datetime(data.file_status_completed_date)
data.decision_date=pd.to_datetime(data.decision_date)
data.deposit_received_date=pd.to_datetime(data.deposit_received_date)
data.response_date=pd.to_datetime(data.response_date)
data.date_acceptance_letter_sent=pd.to_datetime(data.date_acceptance_letter_sent)

In [507]:
# Time Interval Variables

## Decide not to use the "deposit received time" since for those who do not accpet the offer, there will be no deposit
## and there would be too many missing values for the calculations on it.
## Turn to use the boolean variable "deposit_received"

data['time_to_finish_application']=data.file_status_completed_date-data.application_received_date
data['time_to_finish_application']=data['time_to_finish_application'].astype('timedelta64[D]')

data['decision_wait_time']=data.decision_date-data.file_status_completed_date
data['decision_wait_time']=data['decision_wait_time'].astype('timedelta64[D]')

data['response_time']=data.response_date-data.date_acceptance_letter_sent
data['response_time']=data['response_time'].astype('timedelta64[D]')

### Deal With Categorical Variables

In [508]:
factors_to_code=['billing_country','ethnicity','gender','first_generation','campus___cleaned',
'residency','deposit_received','temperature']

In [509]:
# Dummy Code the Categorical Variables
data = pd.get_dummies(data, columns=factors_to_code)

In [510]:
# Remove some Correlated Columns (eg: There will be needed 3 dummy factors for a 4 levels categorical variable )
factors_to_remove=['billing_country_China','ethnicity_African American','gender_Male','first_generation_No',
                  'campus___cleaned_Skirball Academic Center','residency_None','deposit_received_False','temperature_blank']
data.drop(factors_to_remove,axis=1,inplace=True)

In [511]:
# Manually Deal with program_length variable
dictionary={'6 Semester':6,
            '3 years':6,
            '4 years':8,
            '5 Semester':5,
            '8 Semester':8,
            '2 years':4,
            '3 Semester':3,
            '4 Semester':4,
           'blank':np.nan}

In [512]:
data['program_length']=data.program_length.map(dictionary)

In [513]:
## fill the median of program length missing value according to the post-code(program code)
for var in data.post_code.unique():
    df=data[data.post_code==var]
    df.program_length.fillna(df.program_length.median(),inplace=True)
    data[data.post_code==var]=df
# there still missing values since post_code 1632 doese not have any value
# for this issue of Nan, fill the median of of all record
data.program_length.fillna(data.program_length.median(),inplace=True)

/Users/zijianwang/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


### Deal with Numerical Variables

In [514]:
# ther is an outlie in this 'second_review_exp' field, remove this record
data[data.second_review_exp==13] # index is 1626
data.drop(1626,inplace=True)

In [515]:
# Rescale the scores
for i in data.index:
    if data.loc[i,'application_term']=='Spring 2015' or data.loc[i,'application_term']=='Fall 2015':
        data.loc[i,'first_review_sop']=data.loc[i,'first_review_sop']/7*5
        data.loc[i,'first_review_lor']=data.loc[i,'first_review_lor']/3*5
        data.loc[i,'second_review_sop']=data.loc[i,'second_review_sop']/7*5
        data.loc[i,'second_review_lor']=data.loc[i,'second_review_lor']/3*5

### Get the columns that will be used for Modeling

In [516]:
# For the output variable, we need to dummy code it, if comfirmed then 1, 
for i in data.index:  
    if data.loc[i,'response']=='Confirmed':
        data.loc[i,'response']=1
    else:
        data.loc[i,'response']=0

In [517]:
### Final Data Cleaning and Missing Values Filling

# To fill the missing values in time interval variables
for var in data.post_code.unique():
    df=data[data.post_code==var]
    df.time_to_finish_application.fillna(df.time_to_finish_application.median(),inplace=True)
    df.response_time.fillna(df.response_time.median(),inplace=True)
    df.decision_wait_time.fillna(df.decision_wait_time.median(),inplace=True)
    data[data.post_code==var]=df

/Users/zijianwang/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [518]:
## Still Suffer the post_code 1632 issue
## Same method to deal with that
data.time_to_finish_application.fillna(data.time_to_finish_application.median(),inplace=True)
data.response_time.fillna(data.response_time.median(),inplace=True)
data.decision_wait_time.fillna(data.decision_wait_time.median(),inplace=True)

In [519]:
## To fill the 'distance' variables
data[['90041_mi','91601_mi','92122_mi','94607_mi']].describe()

,90041_mi,91601_mi,92122_mi,94607_mi
count,16757.000000,16757.000000,16757.000000,16757.000000
mean,822.588764,826.441366,853.791124,998.731859
std,964.225819,966.893129,927.003138,936.039176
min,0.000000,0.000000,0.000000,0.000000
25%,22.139545,26.101987,99.376021,343.704943
50%,315.591581,307.900092,400.906528,400.036402
75%,1795.713664,1804.814307,1756.997274,1952.588262
max,7809.398124,7818.104716,7778.663825,7943.748647


In [520]:
## If a student is not a US Citizen or PR, then will give it 7800 miles as distance
## If a student is a US Citizen or PR, then input the mean of the field
for i in data.index:
    if (data.loc[i,'residency_Permanent US Resident']==1 or data.loc[i,'residency_US Citizen']==1) and pd.isna(data.loc[i,'90041_mi'])==True:
        data.loc[i,'90041_mi']=822.588764
        data.loc[i,'91601_mi']=826.441366
        data.loc[i,'92122_mi']=927.003138
        data.loc[i,'94607_mi']=998.731859
        
    elif data.loc[i,'residency_Permanent US Resident']!=1 and data.loc[i,'residency_US Citizen']!=1 and pd.isna(data.loc[i,'90041_mi'])==True:
        data.loc[i,'90041_mi']=7800
        data.loc[i,'91601_mi']=7800
        data.loc[i,'92122_mi']=7800
        data.loc[i,'94607_mi']=7800   

In [521]:
# Create a new variable indicating whether the student is for MSW program
# from code book: Post code of student -meaning program of study - 
# 1411 is virtual MSW, 272 is PhD, 1632 is Nursing, 1645 is DSW, 
# the rest are campus MSW with the majority being 264.
msw_dict={
    '1411':'MSW',
    '264':'MSW',
    '1632':'Nursing',
    '1645':'DSW',
    'blank':'blank',
    '1447':'MSW',
    '272':'PhD',
    '797':'MSW',
    '1130':'MSW',
    '1435':'MSW',
    '1397':'MSW',
    '1222':'MSW',
    '1138':'MSW'}
data['program_name']=data.post_code.map(msw_dict)

### The dataset is all set and will output the big dataset and a dataset only with target students and all numerical variables

In [522]:
# Output the complete dataset for future reference
data.to_csv('data_all.csv',index=False)

In [523]:
# Prepare a dataset that is ready for modeling
df=data[(data.decision=='1st Tier Admit') & (data.program_name=='MSW')]

In [524]:
columns=['undergraduate_gpa', 'self_reported_gpa', 'first_review_exp',
       'first_review_sop', 'first_review_gpa', 'first_review_lor',
       'second_review_exp', 'second_review_sop', 'second_review_gpa',
        'distance_to_campus',
       'program_length', 'response',
       'academic_condition', 'age', '90041_mi', '91601_mi', '92122_mi', '94607_mi',
       'last_60_90_gpa', 'graduate_gpa', 'second_review_lor', 'sip_score',
       'time_to_finish_application', 'decision_wait_time', 'response_time',
       'billing_country_United States', 'billing_country_blank',
       'ethnicity_Arab/Middle Eastern', 'ethnicity_Asian/Pacific Islander',
       'ethnicity_Caucasian', 'ethnicity_Latino/a', 'ethnicity_Multi-Ethnic',
       'ethnicity_Native American', 'ethnicity_Unknown', 'ethnicity_blank',
       'gender_Female', 'gender_Other', 'gender_blank', 'first_generation_Yes',
       'first_generation_blank',
       'campus___cleaned_Orange County Academic Center',
       'campus___cleaned_San Diego Academic Center',
       'campus___cleaned_University Park Campus',
       'campus___cleaned_Virtual Academic Center', 'campus___cleaned_blank',
       'residency_International', 'residency_Non Resident', 'residency_Other',
       'residency_Permanent US Resident', 'residency_Temporary US Resident',
       'residency_US Citizen', 'residency_blank', 'deposit_received_True',
       'temperature_Cold', 'temperature_Hot', 'temperature_No Contact',
       'temperature_Warm']

In [525]:
df=df[columns]

In [526]:
df['academic_condition']=df.academic_condition.apply(lambda x: 1 if x==True else 0)

In [527]:
df.to_csv('data_for_modeling.csv',index=False)